##### 找个一个网站的根URL，然后爬取该网站下所有网页的URL

In [1]:
import requests
from bs4 import BeautifulSoup
from queue import Queue
import time
from tqdm import tqdm
from urllib.parse import urljoin

In [2]:
proxies = {
  'http': 'http://127.0.0.1:7890',
  'https': 'http://127.0.0.1:7890',
}

In [ ]:
requests.DEFAULT_RETRIES = 5
s=requests.session()
# 关闭多余连接
s.keep_alive=False

base_url = "xxx"

In [ ]:
def judge(url): # 判断该URL是否应该保存， 自己根据网站的格式来写规则
    return url.startswith(base_url)

In [ ]:
q = Queue()
q.put(base_url)
res = set()
res.add(base_url)
bar = tqdm(total=10000)
while not q.empty():
    url = q.get()
    try:
        content = s.get(url, proxies=proxies)
    except:
        q.put(url)
        time.sleep(1)
        continue
        
    if content.ok:
        soup = BeautifulSoup(content.text)
        response = soup.select('a')
        lists = []
        for i in range(len(response)):
            try:
                lists.append(response[i]['href'])
            except:
                pass
        lists = [i for i in lists if judge(i)]
        # lists = [urljoin(base_url, i) for i in lists if judge(i)]
        lists = list(set(lists))
        for l in lists:
            if l not in res:
                res.add(l)
                q.put(l)
    else:
        res.remove(url)
    bar.set_description("len(set) = {}".format(len(res)))
    bar.update()
    time.sleep(1)

In [ ]:
res = list(res)
fp = open("xxx.txt", 'w', encoding='utf-8')
for i in res:
    fp.write(i + '\n')
fp.close()